In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [57]:
plt.rc('figure',dpi=100);

In [82]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [83]:
training_labels = training_labels[['person','label']]
#data = data.drop(columns = {'Email','Unknown','Social','days_since_last_session',})
#data = data[['person', 'checkout', 'conversion', 'generic listing', 'search engine hit', 'searched products','staticpage', 'promedio dias', 'visited site','retornos', 'total_time', 'max_session_time','sessions','mean_time_by_session', 'viewed product','max_events_by_session']]
#data = data[['person', 'ad campaign hit','Email', 'Organic', 'Paid', 'Social', 'brand listing', 'checkout', 'conversion', 'generic listing', 'search engine hit', 'searched products', 'staticpage', 'viewed product', 'visited site', 'promedio dias', 'retornos', 'longitud busqueda', 'total_time', 'max_session_time', 'mean_events_by_session', 'days_since_last_session']]

In [84]:
data.shape

(38829, 37)

In [85]:
#training_labels = training_labels.head(1960)
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [86]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [87]:
RANDOM_SEED = 123
train, test = train_test_split(features,test_size=0.10,random_state=RANDOM_SEED)
print("Train: ",len(train),"Test: ",len(test))
features=list(features.columns)
features.remove('person')
features.remove('label')

x_train=train[features]
y_train=train['label']

x_test=test[features]
y_test=test['label']

Train:  1764 Test:  196


In [88]:
def grid_search():
    n_estimators = [20,30,40,50,60,70,80,90,100]
    min_samples_split = [10,15,20,25,30,35,40,45,50]
    n_jobs = [-1]
    maximos = []
    max_score = 0
    for n_est in n_estimators:
        for min_sam in min_samples_split:
            for n_j in n_jobs:
                c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
                dt=c.fit(x_train,y_train)
                score=c.score(x_test,y_test)*100
                if max_score < score:
                    maximos = [n_est,min_sam,n_j]
                    max_score = score
                #print("n_estimators : "+ str(n_est)+"; min_samples_split: "+str(min_sam)+"; n_job: "+str(n_j)+"; SCORE: "+ str(score))
    print("El mejor score fue: "+str(max_score))
    return maximos

In [89]:
mejores_hiperparmetros = grid_search()
mejores_hiperparmetros

El mejor score fue: 80.10204081632652


[70, 35, -1]

In [90]:
c=RandomForestClassifier(n_estimators=mejores_hiperparmetros[0],min_samples_split=mejores_hiperparmetros[1]
                         ,n_jobs=mejores_hiperparmetros[2],random_state=0,class_weight="balanced",criterion = 'entropy')
dt=c.fit(x_train,y_train)

In [91]:
score=c.score(x_test,y_test)*100
score

80.10204081632652

In [92]:
prediccion=pd.merge(labels_predict,data,on='person',how='inner')
prediccion = prediccion.fillna(0)

In [93]:
labels_predict.shape

(19415, 1)

In [94]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

array([0, 1, 0, ..., 1, 1, 1])

In [95]:
prediccion['label']=y_final
prediccion.head()

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,max_events_by_session,days_since_last_session,promedio de TFIdf,different_products_views,total_views,same_product_max_views,viewed/checkouted min,viewed/checkouted max,viewed/checkout gral,label
0,4886f805,0.0,0.0,0.0,1.0,0.0,0.0,0,0,1,...,9,186,1.000000,2.0,4.0,3.0,1.0,0.0,2.0,0
1,0297fc1e,17.0,0.0,56.0,1.0,0.0,0.0,29,4,7,...,18,176,1.146942,105.0,404.0,45.0,1.0,0.0,21.0,1
2,2d681dd8,0.0,0.0,0.0,1.0,0.0,0.0,1,5,1,...,16,177,1.331171,10.0,13.0,3.0,1.0,0.0,10.0,0
3,cccea85e,1.0,0.0,1.0,14.0,0.0,0.0,15,7,1,...,104,173,1.375124,188.0,739.0,34.0,1.0,0.0,188.0,1
4,4c8a8b93,9.0,0.0,2.0,3.0,0.0,0.0,14,8,2,...,51,182,1.303777,69.0,177.0,10.0,1.0,0.0,34.5,1


In [96]:
prediccion['label'].value_counts()

0    13356
1     6059
Name: label, dtype: int64

In [97]:
prediccion[['person', 'label']].to_csv('RandomForestResults.csv',index=False)

In [98]:
# Get numerical feature importances
importances = list(c.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: checkout Importance: 0.13
Variable: viewed/checkout gral Importance: 0.11
Variable: days_since_last_session Importance: 0.09
Variable: viewed/checkouted min Importance: 0.06
Variable: total_time Importance: 0.05
Variable: viewed product Importance: 0.04
Variable: promedio dias Importance: 0.04
Variable: max_session_time Importance: 0.04
Variable: different_products_views Importance: 0.04
Variable: total_views Importance: 0.04
Variable: brand listing Importance: 0.03
Variable: promedio hora Importance: 0.03
Variable: mean_time_by_session Importance: 0.03
Variable: max_events_by_session Importance: 0.03
Variable: generic listing Importance: 0.02
Variable: search engine hit Importance: 0.02
Variable: visited site Importance: 0.02
Variable: sessions Importance: 0.02
Variable: mean_events_by_session Importance: 0.02
Variable: same_product_max_views Importance: 0.02
Variable: viewed/checkouted max Importance: 0.02
Variable: Direct Importance: 0.01
Variable: Organic Importance: 0.01